# Welcome to FOSSIL!

            ..\\\\
         ..\\\\  ,\
    --===)))))))   )
         ''////  `/
            ''////  

                                                                                                                   
## Overview
This tool uses neural network templates alongside the CEGIS architecture to automatically synthesise sound Lyapunov functions of Barrier Certificates for any N-dimensional system. 

To use this tool please change the python code in the corresponding cells to input your required dynamical system and domain, then use the correspoding cell to synthise a [Lyapunov function](#lyap-synth) of [Barrier function](#barrier-synth) respectively.

If you would like access to more settings, such as , please use the [advanced-playground](FOSSIL-advanced-playground.ipynb). To see benchmarks and further examples refer to [benchmarks](FOSSIL-benchmarks.ipynb).

In [ ]:
# % Imports
import sys

sys.path.append('..')
from experiments.playground_utils import *
#sys.path.append(".")

### Synthesising Lyapunov Functions

First, we demonstrate synthesising Lyapunov functions. Consider the following dynamical system:

$$ \begin{cases} \dot{x} = -x + xy \\ \dot{y} = - y \end{cases}$$


This system can be instantied as follows, with $x_0 = x $ and $x_1 = y$:


In [ ]:

N_Dimensions = 2

x, y = initialise_states(N_Dimensions)

dynamics = [
    -x + x * y,
    -y
]

# This will work for any arbitrary variable name, but functions will still be printed as f(x0,...,xn)

This system has an equilibrium point at the origin $\mathbf{x} = 0$. We seek to synthesis a Lyapunov function to serve as a certificate of stability of this equilibrium point. This  Lyapunov function will be validated in a region around the equilibrium point. This domain is defined as:

$$ \mathcal{D}  = \{\mathbf{x}: \mathbf{x} < r^2 \} $$

for some user-set parameter r. To constrain the domain to a torus shape (i.e., excluding a smaller hypershere around the origin), use the keyword arguement INNER_RADIUS. 
<a id='lyap-synth'></a>

In [ ]:
r = 10
mode = PrimerMode.LYAPUNOV
numerical_V, symbolic_V = synthesise(dynamics, mode, OUTER_RADIUS=r)

In [ ]:
# The symbolic function can be printed using print_f
print_f(symbolic_V)

### Barrier Cerificates

## What would a good example for here be? Ideally one that is robust to seeds and can be expressed as Spheres and rectangles?

We now demonstrate synthesising a barrier certificate. Consider the following dynamical system:

$$\begin{cases}
\dot{x} = e^{-x} + y - 1 \\
\dot{y} = - \sin^2 x
\end{cases}$$

We can initialise this dynamical system as follows, with with $x_0 = x $ and $x_1 = y$.

In [ ]:
N_Dimensions = 3

x0, x1, x2 = initialise_states(N_Dimensions)

dynamics = [
    x1,
    x2,
    4180*x2 - 2400*x1 - 576*x0
    
]

#### Domain Initialisation
We define the overall domain as 
$$\begin{align*}
& X = \{-2 \leq x, -5 \leq y, \ x \leq 5, \ y  \leq 2 \} \\ 
& X_0 = \{(x+0.5)^2+(y - 0.5)^2 \leq 0.16 \} \\ 
& X_u  = \{ (x - 0.7)^2+(y+0.7)^2 \leq 0.09 \}
\end{align*}$$


This can be initialised as follows:

In [ ]:
xd = Sphere([0,0,0], 2)
xi = Sphere([1, 1, 1,], 0.5)
xu = Sphere([-2, -2, -2,], 0.4)

A barrier function is then synthesised as:
<a id='barrier-synth'></a>

In [ ]:
mode = PrimerMode.BARRIER

f_n, f_v, = synthesise(dynamics, mode, XD=xd, XI=xi, XU=xu, 
                       CEGIS_PARAMETERS={CegisConfig.VERIFIER.k:VerifierType.DREAL,
                                         CegisConfig.ACTIVATION.k: [ActivationType.LINEAR]})